In [1]:
import torch
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import requests
import json
from nltk.corpus import brown
from torch.utils.data import Dataset, DataLoader
import random
import io
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from collections import Counter
import torch.nn.functional as F
import requests

download and prepare the Brown corpus from NLTK for training a Word2Vec model, which is used for creating embeddings of words.

Download Squad2 dataset, split it into training and testing sets, and save them to files.

In [2]:
# Download SQuAD2 dataset
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
response = requests.get(url)

#Save the dataset to a file
dataset_file = "squad2.json"
with open(dataset_file, 'wb') as f:
   f.write(response.content)

#Split dataset into train and test files
train_ratio = 0.8
train_file = "train.json"
test_file = "test.json"

with open(dataset_file, 'r') as f:
    squad_data = json.load(f)
    data = squad_data['data']

    # Shuffle the data
    random.shuffle(data)

    # Split into train and test sets
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]

    # Save train data to file
    train_squad_data = {'data': train_data}
    with open(train_file, 'w') as train_f:
        json.dump(train_squad_data, train_f)

    # Save test data to file
    test_squad_data = {'data': test_data}
    with open(test_file, 'w') as test_f:
        json.dump(test_squad_data, test_f)

In [3]:
#We will use this function on each file in the dataset - test and train
#only the filename is required as the previous function returns the specified file to the same location

def extract_squad_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        file_name = json.load(f)

    paragraphs = []
    for article in file_name['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer['text'] for answer in qa['answers']]
                for answer in answers:
                    paragraphs.append((question, answer))

    df = pd.DataFrame(paragraphs, columns=['Question', 'Answer'])
    return df

In [4]:
#convert .json files to dataframes
train_df = extract_squad_data("train.json") 
test_df = extract_squad_data("test.json")


In [5]:


import pandas as pd
import nltk
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(df):
    cleaned_df = df.copy()
    for column in cleaned_df:
        cleaned_df[column] = cleaned_df[column].apply(lambda sentence: process_sentence(sentence))
    return cleaned_df

def process_sentence(sentence):
    sentence = ''.join([s.lower() for s in sentence if s not in string.punctuation])
    sentence = ' '.join(stemmer.stem(w) for w in sentence.split())
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sentence)

    return ' '.join(tokens) 
# usage:
# df = clean_text(df)


In [6]:
train_clean_df = clean_text(train_df)


In [7]:
print(train_clean_df.head())

                                            Question                    Answer
0        the time is base in what major british citi                    london
1   what year did the initi incarn of the time start                      1785
2  what was the origin name of the time befor cha...  the daili univers regist
3  the time own by news uk is itself own by what ...           news corp group
4  the time and it sister paper the sunday time a...              time newspap


In [8]:
class Vocab:
    def __init__(self):
        self.word2index = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 4: "<UNK>"}
        self.word2count = {}
        self.n_words = 4  # Count <PAD>, <SOS>, <EOS>, <UNK>
    def build_vocab(self, sentences):
        for sentence in sentences:
            for word in self.tokenize(sentence):
                self.add_word(word)
    def tokenize(self, sentence):
        return sentence.strip().split()
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [9]:

""""

    Tokenizes English text from a string into a list of strings (tokens)
 
    return [tok.text for tok in spacy_en.tokenizer(text)]

   """

'"\n\n    Tokenizes English text from a string into a list of strings (tokens)\n \n    return [tok.text for tok in spacy_en.tokenizer(text)]\n\n   '

In [10]:
question_vocab = Vocab()
answer_vocab = Vocab()


questions = train_clean_df['Question']
answers = train_clean_df['Answer']

question_vocab.build_vocab(questions)
answer_vocab.build_vocab(answers)

question_vocab.add_word('<UNK>')
answer_vocab.add_word('<UNK>')



In [11]:
question_vocab_size = len(question_vocab.word2count)
print(question_vocab_size)

25141


In [12]:
answer_vocab_size = len(answer_vocab.word2count)
print(answer_vocab_size)

28629


In [13]:
print(question_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'the', 5: 'time', 6: 'is', 7: 'base', 8: 'in', 9: 'what', 10: 'major', 11: 'british', 12: 'citi', 13: 'year', 14: 'did', 15: 'initi', 16: 'incarn', 17: 'of', 18: 'start', 19: 'was', 20: 'origin', 21: 'name', 22: 'befor', 23: 'chang', 24: 'it', 25: 'own', 26: 'by', 27: 'news', 28: 'uk', 29: 'itself', 30: 'corpor', 31: 'and', 32: 'sister', 33: 'paper', 34: 'sunday', 35: 'are', 36: 'publish', 37: 'compani', 38: 'from', 39: 'daili', 40: 'univers', 41: 'regist', 42: 'india', 43: 'strait', 44: 'singapor', 45: 'new', 46: 'york', 47: 'irish', 48: 'cape', 49: 'south', 50: 'africa', 51: 'typefac', 52: 'font', 53: 'who', 54: 'creator', 55: 'use', 56: 'novemb', 57: '2006', 58: 'to', 59: 'how', 60: 'long', 61: 'print', 62: 'broadsheet', 63: 'format', 64: 'a', 65: 'compact', 66: 'size', 67: 'tradit', 68: 'polit', 69: 'spectrum', 70: 'support', 71: 'parti', 72: '2001', 73: '2005', 74: 'general', 75: 'elect', 76: 'march', 77: '2014', 78: 'had', 79: 'an', 80: 'av

In [14]:
print(answer_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'london', 5: '1785', 6: 'the', 7: 'daili', 8: 'univers', 9: 'regist', 10: 'news', 11: 'corp', 12: 'group', 13: 'time', 14: 'newspap', 15: '1788', 16: '1838', 17: '1845', 18: '1851', 19: '1859', 20: '1872', 21: 'roman', 22: 'stanley', 23: 'morison', 24: 'modern', 25: '219', 26: 'year', 27: '2004', 28: 'moder', 29: 'labour', 30: 'parti', 31: '394448', 32: 'gage', 33: 'cengag', 34: 'conserv', 35: 'john', 36: 'walter', 37: 'libel', 38: '940', 39: '5000', 40: 'steamdriven', 41: 'cylind', 42: 'press', 43: 'reput', 44: 'friedrich', 45: 'koenig', 46: 'thoma', 47: 'barn', 48: 'his', 49: 'son', 50: 'thunder', 51: 'rotari', 52: 'print', 53: 'thadeus', 54: 'delan', 55: 'war', 56: 'w', 57: 'h', 58: 'russel', 59: 'crimean', 60: 'irish', 61: 'potato', 62: 'famin', 63: 'great', 64: 'reform', 65: 'bill', 66: 'of', 67: '1832', 68: 'corn', 69: 'law', 70: 'wealthi', 71: 'class', 72: 'secessionist', 73: '1847', 74: 'penni', 75: 'foreign', 76: 'offic', 77: '19th', 78:

In [15]:
def pad_or_cut(df, column_names, length):
    pad_symbol = "<PAD>"
    
    for column_name in column_names:
        df[column_name] = df[column_name].apply(lambda x: (x.split()[:length] + [pad_symbol]*length)[:length])
        df[column_name] = df[column_name].apply(lambda x: ' '.join(x))
    return df


In [16]:

padded_df = pad_or_cut(train_clean_df, ['Question', 'Answer'], 10)

In [17]:
def words_to_indices(df, question_vocab, answer_vocab):
    df['Question'] = df['Question'].apply(lambda sentence: [question_vocab.word2index[word] if word in question_vocab.word2index else question_vocab.word2index['<UNK>'] for word in sentence.split()])
    df['Answer'] = df['Answer'].apply(lambda sentence: [answer_vocab.word2index[word] if word in answer_vocab.word2index else answer_vocab.word2index['<UNK>'] for word in sentence.split()])
    return df

In [18]:
idx_df = words_to_indices(padded_df, question_vocab, answer_vocab) # could have used nn.padding & nn.embedding

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
idx_df.head(20)

from torch.utils.data import TensorDataset, DataLoader

In [21]:
trainlist_src = idx_df['Question'].tolist() #extract index seqences from df
trainlist_trg = idx_df['Answer'].tolist()

In [22]:
QA_pairs = [(src, trg) for src, trg in zip(trainlist_src, trainlist_trg) if len(src) == len(trg)]

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
train_src = [pair[0] for pair in QA_pairs]
train_trg = [pair[1] for pair in QA_pairs]

In [25]:
source_tensor = torch.tensor(train_src, dtype=torch.long) 
target_tensor = torch.tensor(train_trg, dtype=torch.long)

In [26]:
source_tensor.shape[0]


69459

In [27]:
target_tensor.shape[0]

69459

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, x, hidden, cell_state):
        x = self.embedding(x)
        x = x.view(len(x), 1, -1)  # Adjust for sequence length
        x, (hidden, cell_state) = self.lstm(x, (hidden, cell_state))
        return x, hidden, cell_state



In [29]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, self.hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden, cell_state):
        x = self.embedding(x)
        x = x.view(len(x), 1, -1)  # Adjust for sequence length
        x, (hidden, cell_state) = self.lstm(x, (hidden, cell_state))
        x = self.softmax(self.fc(x[0]))
        return x, hidden, cell_state

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder = Encoder(self.input_size, self.hidden_size)
        self.decoder = Decoder(self.hidden_size, self.output_size)
        
    def forward(self, source_tensor, target_tensor, teacher_forcing_ratio=0.5):
        src_len = source_tensor.shape[0]
        trg_len = target_tensor.shape[0]
        batch_size = target_tensor.shape[1]
        trg_vocab_size = self.output_size
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        
        encoder_hidden = torch.zeros([1, 1, self.hidden_size]).to(device)
        cell_state = torch.zeros([1, 1, self.hidden_size]).to(device)
        
        for i in range(src_len):
            encoder_output, encoder_hidden, cell_state = self.encoder(source_tensor[i], encoder_hidden, cell_state)
            
        decoder_input = torch.Tensor([[0]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoder_cell_state = cell_state

        for t in range(trg_len):
            decoder_output, decoder_hidden, decoder_cell_state = self.decoder(decoder_input, decoder_hidden, decoder_cell_state)
            outputs[t] = decoder_output
            top1 = decoder_output.argmax(1)
            decoder_input = top1 if random.random() < teacher_forcing_ratio else target_tensor[t]
            assert t < len(target_tensor), f"Index out of bounds: Index is {t} but trg length is {len(target_tensor)}"

        
        return outputs
        
    

In [31]:
import torch.nn as nn

In [32]:
import matplotlib.pyplot as plt

In [33]:
question_vocab_size = len(question_vocab.word2count)
answer_vocab_size = len(answer_vocab.word2count)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:


def create_model(input_size, hidden_size, output_size):
    # Instantiate the Seq2Seq model
    seq2seq = Seq2Seq(input_size, hidden_size, output_size).to(device)

    return seq2seq


In [36]:
from torch.utils.data import TensorDataset, DataLoader

In [37]:
#print("Input Tensor Shape:", source_tensor.shape)
#print("Target Tensor Shape:", target_tensor.shape)

In [38]:
dataset = TensorDataset(source_tensor, target_tensor)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
def train_model(seq2seq, dataloader, optimizer, criterion, device):
    seq2seq.train()  # Ensure the network is in train mode
    total_loss = 0

    for source_tensor, target_tensor in dataloader:
        # Send tensors to the correct device
        source_tensor = source_tensor.to(device)
        target_tensor = target_tensor.to(device)

        optimizer.zero_grad()  # Reset the gradients

        output = seq2seq(source_tensor, target_tensor)

        # Adjust the shapes of output and target_tensor for the loss function
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        target_tensor = target_tensor[1:].view(-1)

        loss = criterion(output, target_tensor)  # Compute the loss

        loss.backward()  # Perform backpropagation

        torch.nn.utils.clip_grad_norm_(seq2seq.parameters(), 1)  # Clip the gradients to prevent explosion

        optimizer.step()  # Update the weights

        total_loss += loss.item()  # Accumulate the loss

    return total_loss / len(dataloader)  # Return the average loss



In [41]:
#Hyperparameters

learning_rate = 0.001
num_epochs = 10


In [42]:
input_size = question_vocab_size
hidden_size = 128
output_size = answer_vocab_size
num_epochs = 10


In [43]:
seq2seq = create_model(input_size, hidden_size, output_size)
seq2seq.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25141, 128)
    (lstm): LSTM(128, 128)
  )
  (decoder): Decoder(
    (embedding): Embedding(28629, 128)
    (lstm): LSTM(128, 128)
    (fc): Linear(in_features=128, out_features=28629, bias=True)
    (softmax): LogSoftmax(dim=1)
  )
)

In [44]:
input_size = question_vocab_size
hidden_size = 128
output_size = answer_vocab_size
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:




learning_rate = 0.001
num_epochs = 10
optimizer = torch.optim.SGD(seq2seq.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    loss = train_model(seq2seq, dataloader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [6,0,0

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
